<br>
<h1 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Tamil Nadu - Elections 2021  <br> Data Analysis & Trends </h1>
<br>

### Context

Tamil Nadu 2021 State Assembly Elections - Dataset contains Constituency wise contestant, polling details & Winning details 

### Content

Tamil Nadu 2021 State Assembly Elections - Event date       -6-April-2021
                                                                           Results out on - 2 May-2021

Data File - Tamil_Nadu_State_Elections_2021_Constituency_Metadata.csv

* Constituency - Demographic location name in Tamil Nadu ( India )
* District           - District on which Constituency belongs to ( Probably District may contain 6-8 Constituency )
* Reserved.      -  Holds it belongs to SC/ST or General 
* Lok_sabha_constituency - Lok Sabha is a national election constituency ( Probably each Lok Sabha constituency have 6 State Constituency )
* State_Name    - State Name in India - "Tamil Nadu"

Data File - Tamil_Nadu_State_Elections_2021_Details.csv

* Constituency	- Demographic location name in Tamil Nadu ( India )
* Candidate	        -  Name of the Candidate who contested in the Constituency 
* Party	                - Name of the party the Candidate belongs to 
* EVM_Votes	        - No. of Electoral Voting Machine Votes 
* Postal_Votes	- No. of Votes from Postal
* Total_Votes	        - Total Votes for the candidate 
* %_of_Votes	        - Percentage of votes the candidate get in his constituency 
* Tot_Constituency_votes_polled	- Total no. of votes polled in the constituency 
* Tot_votes_by_parties	                - Total votes the party got in all the constituency 
* Winning_votes	  - Total votes declated as win 
* Win_Lost_Flag  🎊 - True or False 

General Information 👍 -

Total State Election Constituency in Tamil Nadu          - 234 
Total Lok Sabha Election Constituency in Tamil Nadu - 39 

# Contents

* [<font size=4>EDA</font>](#1)
    * [Data Preperation](#1.1)
    * [Univariate Analysis](#1.2)
    * [Bivariate Analysis](#1.3)


In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q sweetviz
!pip install -q klib

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import klib

## Data Vizualisation library
import plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot
from plotly.offline import iplot


import seaborn as sns #for making plots
import matplotlib.pyplot as plt # for plotting
import os  # for os commands
plt.rcParams['figure.figsize'] = [16, 10]
%matplotlib inline

# Data Processing

In [ ]:
df_details = pd.read_csv('/kaggle/input/d/praveengovi/tamil-nadu-2021-state-elections/Tamil_Nadu_State_Elections_2021_Details.csv')
df_cons = pd.read_csv('/kaggle/input/d/praveengovi/tamil-nadu-2021-state-elections/Tamil_Nadu_State_Elections_2021_Constituency_Metadata.csv')
df_alliance = pd.read_csv('/kaggle/input/d/praveengovi/tamil-nadu-2021-state-elections/Tamil_Nadu_State_Elections_2021_Alliance.csv')

In [ ]:
df_details.head()

In [ ]:
df_details[['Party','Total_Votes']].drop_duplicates(keep='first').sort_values('Total_Votes',ascending=False)

In [ ]:
votes_by_parties =df_details.groupby(['Party']).agg({'Total_Votes':'sum'}).reset_index().sort_values('Total_Votes',ascending=False).head(40)
df_alliance.rename(columns={'PARTY_FULL_NAME':'Party'},inplace=True)
votes_by_parties1 = votes_by_parties.merge(df_alliance,on='Party',how='left')
votes_by_parties1['ALLIANCE_NAME'] = votes_by_parties1['ALLIANCE_NAME'].fillna('No Alliance')
votes_by_parties1['PARTY_ABBR'] = votes_by_parties1['PARTY_ABBR'].fillna('NA')
votes_by_parties1 = votes_by_parties1.replace('Non-aligned parties','No Alliance')
votes_by_parties1_with_alliance = votes_by_parties1[votes_by_parties1['ALLIANCE_NAME']!='No Alliance'].sort_values(by=['Total_Votes'],ascending=False)
votes_by_parties1_without_alliance = votes_by_parties1[votes_by_parties1['ALLIANCE_NAME']=='No Alliance'].sort_values(by=['Total_Votes'],ascending=False)
votes_by_parties1_without_alliance_head4 =  votes_by_parties1_without_alliance.head(4)
votes_by_parties1_without_alliance_remains = votes_by_parties1_without_alliance[~votes_by_parties1_without_alliance['Party'].isin(votes_by_parties1_without_alliance_head4['Party'])]
votes_by_parties1_without_alliance_remains1=votes_by_parties1_without_alliance_remains.groupby(['ALLIANCE_NAME']).agg({'Total_Votes':'sum'}).reset_index()
votes_by_parties1_without_alliance_remains1['Party']='NA'
votes_by_parties1_without_alliance_remains1['PARTY_ABBR']='NA'
votes_by_parties1_without_alliance_remains1['Total_Constituency']=np.nan
votes_by_parties1_without_alliance_fnl =  pd.concat([votes_by_parties1_without_alliance_head4,votes_by_parties1_without_alliance_remains1[votes_by_parties1_without_alliance_head4.columns]])

In [ ]:
votes_by_parties1_without_alliance_fnl.head()

In [ ]:
df_party_alliance = pd.concat([votes_by_parties1_with_alliance,votes_by_parties1_without_alliance_fnl])
df_party_alliance

In [ ]:
df_details_fnl = df_details.merge(df_party_alliance[['Party', 'ALLIANCE_NAME', 'PARTY_ABBR',
       'Total_Constituency']],on='Party',how='inner')

# Let Start EDA

# Univariate Analysis

In [ ]:
df_details_fnl['Win_Lost_Flag1']='N'
df_details_fnl.loc[df_details_fnl['Win_Lost_Flag'],'Win_Lost_Flag1']='Y'
df_details_fnl = df_details_fnl[[ x for x in df_details_fnl.columns if x not in 'Win_Lost_Flag']]

# A visualization of the number and frequency of categorical features

In [ ]:
klib.cat_plot(df_details_fnl) # returns a visualization of the number and frequency of categorical features

In [ ]:
df_details_fnl[['Constituency','Party','Total_Votes']]

In [ ]:
party_list = df_details_fnl['Party'].unique().tolist()

# Univariate Analysis of Total votes at Party level

In [ ]:
df_prty_det = df_details_fnl[['Constituency','Party','Total_Votes']].drop_duplicates(keep='first')

In [ ]:
def plot_party_level_votes_distribution(lists,df,field_name):
    for l in lists:
        print("Party Name =>",l)
        klib.dist_plot(df[df[field_name]==l])
    return None

In [ ]:
plot_party_level_votes_distribution(party_list,df_prty_det,'Party')

# Univariate Analysis of Total votes at Alliance level

In [ ]:
df_alliance_det = df_details_fnl.groupby(['Constituency','ALLIANCE_NAME']).agg({'Total_Votes':'sum'}).reset_index()

In [ ]:
alliance_list=df_alliance_det['ALLIANCE_NAME'].unique().tolist()

In [ ]:
plot_party_level_votes_distribution(alliance_list,df_alliance_det,'ALLIANCE_NAME')

#### Observation :- 


# Sweetviz - Glimpse of Account

In [ ]:
import sweetviz as sv
#analyzing the dataset
train_report = sv.analyze(df_details_fnl)
#display the report
train_report.show_html()

In [ ]:
train_report.show_notebook()

# Contesting Candidates at Constituency 

In [ ]:
No_of_candidates_by_cons = df_details_fnl.groupby(['Constituency']).agg({'Candidate':'count'}).reset_index().rename(columns={'Candidate':'No_of_Candidates'})

In [ ]:
klib.dist_plot(No_of_candidates_by_cons)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30, 10))
sns.boxplot(No_of_candidates_by_cons['No_of_Candidates'])

#### Observation :- 
  We can clearly see that no. of cadidates in 4 constituencies is abnormal

In [ ]:
fig, ax = plt.subplots(1, figsize=(30, 10))
sns.violinplot(No_of_candidates_by_cons['No_of_Candidates'])

In [ ]:
df_details_fnl

## Unified view of Top 3 parties which got maximum votes

In [ ]:
plt.rcParams['figure.figsize'] = (18.0, 6.0)
bins = 80
plt.hist(df_details_fnl[df_details_fnl['PARTY_ABBR'] == 'DMK']['Total_Votes'], alpha = 0.6, bins=bins, label='DMK')
plt.hist(df_details_fnl[df_details_fnl['PARTY_ABBR'] == 'AIADMK']['Total_Votes'], alpha = 0.8, bins=bins, label='AIADMK')
plt.hist(df_details_fnl[df_details_fnl['PARTY_ABBR'] == 'NTK']['Total_Votes'], alpha = 0.4, bins=bins, label='NTK')
plt.xlabel('length')
plt.ylabel('Total_Votes')
plt.legend(loc='upper right')
#plt.xlim(0,150)
plt.grid()
plt.show()

# Bivariate Analysis

####  PairPlot of Total votes and Percentage of Votes

In [ ]:
df_details_fnl_corr = df_details_fnl[['ALLIANCE_NAME','%_of_Votes','Total_Votes','Postal_Votes','Win_Lost_Flag1']].drop_duplicates(keep='first')

In [ ]:
sns.pairplot(df_details_fnl_corr,hue='ALLIANCE_NAME')

In [ ]:
sns.heatmap(df_details_fnl_corr.corr())

## Total Votes landscapre across Constituency

In [ ]:
fig, ax = plt.subplots(1, figsize=(30, 10))
sns.lineplot(df_details_fnl['Constituency'],df_details_fnl['Total_Votes'],hue=df_details_fnl['ALLIANCE_NAME'])
plt.xlabel('Constituency')
plt.ylabel('Total_Votes')
plt.legend(loc='upper right')
#plt.xlim(0,150)
plt.show()